In [16]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors
import pandas as pd
from random import uniform

import sys
sys.path.append('../')
from database.models import get_db_session, AccidentDBScan, WorkZone, Incidencia

from sklearn.datasets import load_iris
from sklearn import tree

# THE IRIS DATA SET: https://archive.ics.uci.edu/ml/datasets/Iris
iris = load_iris()
print(iris.feature_names)

# elements
iris.data
#class to predict
iris.target

# http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
clf = tree.DecisionTreeClassifier()
clf = clf.fit(iris.data, iris.target)




# plot the decision tree: 
# http://nbviewer.jupyter.org/github/kittipatkampa/python_dev/blob/master/demo_decision_tree_v1.ipynb
from sklearn.externals.six import StringIO  
import pydotplus

# It is necessary to install GraphViz
# http://www.graphviz.org/Download..php


## Extract the decision tree logic from the trained model
dot_data = StringIO() 
tree.export_graphviz(clf, out_file=dot_data,  
                         feature_names=iris.feature_names,  
                         class_names=iris.target_names,  
                         filled=True, rounded=True,  
                         special_characters=True)

## convert the logics into graph
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  

## This will plot decision tree in pdf file
graph.write_jpeg(path="iris.png") 


['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


InvocationException: Program terminated with status: 1. stderr follows: Warning: Could not load "/home/jcepeda/anaconda3/lib/graphviz/libgvplugin_pango.so.6" - file not found
Format: "jpeg" not recognized. Use one of: canon cmap cmapx cmapx_np dot eps fig gv imap imap_np ismap pdf pic plain plain-ext png pov ps ps2 svg svgz tk vml vmlz xdot xdot1.2 xdot1.4
